In [4]:
import random
import numpy as np
import math

In [27]:
def FMM(z, charges, points, precision, r):
    
    M, = charges.shape
    n = math.log(M,4)
    p = int(round(abs(math.log(precision,2))))
    step = (np.max(points) - np.min(points))/ n
    
    near_index = np.logical_and(points>(z-r),points < (z+r))
    #print near_index
    near_influence = near_field(z, charges[near_index],
                                points[near_index])
    
    
    far_index = np.logical_not(near_index)
    #print far_index
    far_influence = far_field(z, charges[far_index] , points[far_index], p )

    print "near field", near_influence
    print "far field", far_influence
    return near_influence + far_influence
    #far_influence = 

    
def precalc_ak(charges,points, p):
    k = np.arange(1,p+1)
    m = len(points)
    k_mat = np.repeat(k,m).reshape((p,m))
    
    z_k = np.power(points,k_mat)
    mult = np.multiply(z_k,-charges)
    div = np.sum(mult, axis=1) / (k)
    return div

def near_field(z,charges,points):
    z_points = abs(z - points)
    z_points = z_points[z_points != 0]
    log_z = np.log(z_points)
    q_z = charges[(z-points) != 0] * log_z
    influ = np.sum(q_z)
    return influ

def far_field(z,charges,points, p):
    a_k = precalc_ak(charges,points,p)
    Q = sum(charges)
    Qlogz = Q*np.log(z)
    k = np.arange(1,p+1)
    zk = np.power(z,k)
    akzk = a_k/zk
    #print "p", p
    #print "points", points
    #print "q", charges
    #print "ak", a_k
    #print "z", z
    #print "zk", zk
    #print "qlogz", Qlogz
    #print "akzk", akzk
    #print "suma",sum(akzk)
    influ = Qlogz + sum(akzk)
    return influ

def max_error(charges, z, r, p):
    c = abs(z/r)
    print c
    A = np.sum(np.apply_along_axis(abs, 0, charges))
    print A
    return (A/(c - 1))*(1/c)**p

In [37]:
np.random.seed(3)
size = 10000
charges = np.random.random(size) - 0.5
points = np.random.rand(size)

zindex = np.random.randint(len(points))

z = points[zindex]+3
precision = 1e-3
print z
r = 0.05
#r = 1000

fmm = FMM(z,charges,points, precision, r)
print fmm

3.30853269459
near field 0.0
far field -32.6045038564
-32.6045038564


In [38]:
r = 10000
nofmm = FMM(z,charges,points, precision, r)
print nofmm

near field -32.6045025298
far field 0.0
-32.6045025298


In [39]:
Error = abs(fmm - nofmm)
print Error

1.32656222718e-06


In [40]:
r = 0.5
p = int(round(abs(math.log(precision,2))))
max_error(charges, z, r, p)

6.61706538919
2500.10623678


2.7656340956959125e-06